# シフト最適化問題

この章ではJijModelingを使って実際にゼロから数理モデルを組んでいくことを行います。

## シフト最適化問題

あなたがシフト最適化問題を数理最適化を使って解くことを依頼されたとしましょう。  
現場では以下のシフト作成をお願いされています。

- 毎日の勤務（昼、夕方、夜）のシフト作成
- 一週間に最低1日の休日、最低1日の昼勤

を入れてください。

In [1]:
import jijmodeling as jm

demand = jm.Placeholder("demand", dim=2)

N = jm.Placeholder("N")
D = demand.shape[0].set_latex("D")
T = demand.shape[1].set_latex("T")

submitted = jm.Placeholder("S", shape=(N, D, T))

x = jm.Binary("x", shape=(N, D, T+1))

i = jm.Element("i", N)
date = jm.Element("d", D)
t = jm.Element("t", T)


problem = jm.Problem("scheduling")
jmC = jm.Constraint
problem += jm.Sum([i, date, t], submitted[i, date, t])
problem += jmC("demand", x[:, date, t] >= demand[date, t], forall=[date, t])

day_week = jm.Element("day_week", (date, date+7)).set_latex("d'")
problem += jmC("holiday", jm.Sum(day_week, x[i, day_week, T]) >= 1, forall=[i, (date, date < D-7)])

problem

In [118]:
import jijzept as jz
import numpy as np

# sampler = jz.JijDA3Sampler(config="config.toml", da3_token="")
instance = {
    "N": 10,
    "demand": np.random.uniform(1, 5, (5, 3)), 
}
# response = sampler.sample_model(problem, instance)

In [119]:
# import jijmodeling.transpiler as jmt

# compiled_model = jmt.core.model_compile(problem, instance, {})
# mip_builder = jmt.core.mip.transpile_to_mip(compiled_model)
# mip_model = mip_builder.get_model()

# status = mip_model.optimize()

# response = mip_builder.decode_from_mip(status, mip_model)

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-15) rows, 0 (-200) columns and 0 (-150) elements
Clp0000I Optimal - objective value 45.596908
Coin0511I After Postsolve, objective 45.596908, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 45.59690834 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00

Starting MIP optimization
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01



In [120]:
x_value = response.to_dense().record.solution["x"][0]

In [123]:
import numpy as np
import pandas as pd

schedules = {_i: x_value[_i].astype(int) for _i in range(instance["N"])}

assigned_number = np.zeros(shape=schedules[0].shape, dtype=int)
for _i, table in schedules.items():
    assigned_number += table
schedule_df = {_i: pd.DataFrame(table) for _i, table in schedules.items()}
schedule_df[0]

,0,1,2,3
0,0,0,1,0
1,1,1,1,0
2,0,1,0,0
3,1,1,1,0
4,0,1,1,0
